<div >
<img src = "figs/ans_banner_1920x200.png" />
</div>

# Clustering II. Sesión Sincrónica.


Tanto Clustering como PCA y SVD buscan simplificar los datos de forma no supervizada, pero sus mecanismos son diferentes:

- PCA y SVD buscan encontrar una representación de baja dimensión de las observaciones que explique una buena fracción de la varianza;

- Clustering busca encontrar subgrupos homogéneos entre las observaciones.


El objetivo de este cuaderno es introducir los algoritmos que serán estudiados en la semana 4.

**NO** es necesario editar el archivo o hacer una entrega. Los ejemplos contienen celdas con código ejecutable (`en gris`), que podrá modificar libremente. Esta puede ser una buena forma de aprender nuevas funcionalidades del *cuaderno*, o experimentar variaciones en los códigos de ejemplo.



# Introducción 

## ¿Qué es el análisis de clusters? 

El análisis de clusters es una de las principales aplicaciones de los algoritmos de aprendizaje no supervisado. Este tipo de análisis se utiliza para juntar observaciones similares en grupos:



<div style="max-width:500px">
<img src = "figs/plot_clustering_notebook.png" />
</div>



## Caveat

Los métodos de clustering son exploratorios: se pueden utilizar para evaluar la calidad de los datos y generar hipótesis. 

Pero no importa lo que entre en el algoritmo de agrupamiento, los clusters salen. Esta es una situación clásica de "basura que entra, basura que sale". 


La conclusión es que la agrupación es buena si es útil para responder el problema en particular. Pero, esto es difícil de evaluar, a pesar de tener ciertas medidas de validez interna. Esto requiere que el usuario utilice su capacidad y discernimiento.


## Distancias

Ejemplo 1: Calcular el parecido entre tres alumnos/as  a partir de sus notas usando la distancia euclideana:

In [ ]:
import pandas as pd
import numpy as np

A= pd.DataFrame(np.array([[3,5,2,4], [1, 0, 3, 5], [9, 10, 2, 5]]))
A

Distancia Euclideana:



\begin{equation}
d_{02} = \sqrt{(3−9)^2 +(5−10)^2 +(2−2)^2 +(4−5)^2} =7,87 \\
d_{12} = \sqrt{(1−9)^2 +(0−10)^2 +(3−2)^2 +(5−5)^2} =12,85  \\
d_{01} = \sqrt{(3−1)^2 +(5−0)^2  +(2−3)^2 +(4−5)^2} =5,57 
\end{equation}



In [ ]:
import math
math.sqrt((3-9)**2 +(5-10)**2 +(2-2)**2 +(4-5)**2 )

In [ ]:
math.sqrt((1-9)**2 +(0-10)**2 +(3-2)**2 +(5-5)**2 )

In [ ]:
math.sqrt((3-1)**2 +(5-0)**2  +(2-3)**2 +(4-5)**2 )

In [ ]:
from scipy.spatial import distance_matrix

# Creamos la matriz de distancias escogiendo p = 2, el cual convierte 
# la distancia Minkowski en la distancia euclidiana
pd.DataFrame(distance_matrix(A, A, p = 2))

- Un problema de la distancia euclídiana, como medida de similaridad, es su dependencia de las diferentes escalas en que estén medidas las variables. 

- Escalas y rangos de variación diferentes pueden afectar al análisis de clusters.

- Este problema se soluciona si en vez de calcular la distancia euclídea con puntuaciones directas se calcula con puntuaciones normalizadas. 

- Estandarizar las puntuaciones de los sujetos en las variables es uno de los procedimientos de normalización más frecuentes en análisis de datos. 

EJEMPLO 2. Supongamos que estamos interesados en agrupar a una muestra de 5 familias en base al número de hijos, al
sueldo en euros al mes y al tamaño de la casa en metros cuadrados. La matriz de datos de la que partimos es:

In [ ]:
B= pd.DataFrame(np.array([[1,723,60], [1, 900, 60], [4,800,80], [0,1205,50], [2,600,65]]))
B.columns = ['Hijos','Salario','Metros2']
B

Podemos como antes calcular las distancias entre los sujetos a partir de las puntuaciones directas o bien podemos calcularlas a partir de las variables estandarizadas.

In [ ]:
pd.DataFrame(distance_matrix(B, B, p = 2))

Como puede observarse, las familias más parecidas son la familia primera y la tercera. Sin embargo, son familias que salvo en que tienen un salario similar son diferentes en el resto de las variables. Si por el contrario seleccionamos la opción estandarizar
la matriz de distancias que obtenemos es:

In [ ]:
from sklearn.preprocessing import scale

pd.DataFrame(distance_matrix(scale(B), scale(B), p = 2))

- Con las puntuaciones estandarizadas las familias más parecidas son la primera y la segunda. 

- Es evidente que los resultados de un análisis de clusters son distintos si se parte de matrices de similaridad o distancia que ordenen a los sujetos de manera distinta. 

- Es por ello que en caso de variables medidas en escalas distintas es necesario normalizar.

EJEMPLO 3.  Supongamos que queremos agrupar a los sujetos de una muestra (N = 5) en
función de su parecido en un conjunto de variables todas ellas dicotómicas: 
- Estado civil: soltero(1)-casado(0)
- Situación laboral: ocupado(1)-desocupado(0)
- Nivel de estudios: bajo(1)-alto(0)
- Creencias religiosa: creyente(1)-no creyente(0)
- Tendencia de voto en las últimas elecciones: izquierda(1)-derecha(0). 

La matriz de datos de la que partimos es

In [ ]:
C= pd.DataFrame(np.array([[1,1,0,1,0], [1,1,1,0,0], [0,0,0,1,1], [0,0,0,0,1], [1,0,0,1,0]]))
C.columns = ['Soltero','Ocupado','Baja_educacion', 'Creyente', "Izquierda"]
C

 La medida de distancia más utilizada en estos casos se conoce como distancia de coincidencia simple:


\begin{align}
d_{ii'}=d(x_i,x_{i'})= I(x_i \neq x_{i'})
\end{align}

donde $I(.)$ es la función indicadora que toma valor 1 cuando las variables no coinciden, y 0 en caso contrario.

In [ ]:
C.iloc[:,0:1]

In [ ]:
from sklearn.metrics import DistanceMetric

DistanceMetric.get_metric('matching').pairwise(C.iloc[:,0:1])

In [ ]:
DistanceMetric.get_metric('matching').pairwise(C)

EJEMPLO 4. Supongamos que tenemos los siguientes datos

In [ ]:
import pandas as pd

# Creamos un diccionario
dictionary = {"Edad": [22, 25, 30, 38, 42, 47, 55, 62, 61, 90], 
              "Genero": ["M", "M", "F", "F", "F", "M", "M", "M", "M", "M"], 
              "Estado_Civil": ["Soltero", "Soltero", "Soltero", "Casado", "Casado", "Soltero", "Casado", "Divorciado", "Casado", "Divorciado"], 
              "Salario": [18000, 23000, 27000, 32000, 34000, 20000, 40000, 42000, 25000, 70000], 
              "tiene_hijos": [False, False, False, True, True, False, False, False, False, True], 
              "Volumen_compras": ["Bajo", "Bajo", "Bajo", "Alto", "Alto", "Bajo", "Medio", "Medio", "Medio", "Bajo"]}

# Creamos un Pandas DataFrame 
D = pd.DataFrame.from_dict(dictionary)
D

Distancia de Gower

- Para una característica numérica, la diferencia parcial entre dos clientes i y j es la resta entre sus valores en la característica específica (en valor absoluto) dividida por el rango total de la característica. El rango de salario es 52000 (70000–18000) mientras que el rango de edad es 68 (90–22). 
    Note, hay que tener en cuenta si existen outliers o valores atípicos. Un valor erróneo extremadamente grande o pequeño afectaría directamente el rango y, por lo tanto, las diferencias en esa característica, distorsionando su importancia.

-    Para una característica categórica, la diferencia parcial entre dos clientes es uno cuando ambos clientes tienen un valor diferente para esta característica. Cero en caso contrario.
  

In [ ]:
D.iloc[0:2,:]

In [ ]:
D.iloc[:,0].max()-D.iloc[:,0].min()

In [ ]:
abs(22-25)/68

In [ ]:
D.iloc[:,3].max()-D.iloc[:,3].min()

In [ ]:
abs(18000-23000)/52000

La Disimilitud de Gower entre ambos clientes es el promedio de disimilitudes parciales a lo largo de las diferentes características: 

\begin{align}
\frac{(0,044118 + 0 + 0 + 0,096154 + 0 + 0)}{ 6} = 0,023379. 
\end{align}

Como el valor es cercano a cero, podemos decir que ambos clientes son muy similares.

In [ ]:
import gower

distance_matrix = gower.gower_matrix(D)
pd.DataFrame(distance_matrix)

## Clustering Jerárquico

#### Enlace completo (complete linkage - CL)
   
   El enlace completo o técnica del vecino más lejano, es lo opuesto al enlace simple y combina los clusters encontrando la distancia máxima entre las observaciones del cluster $G$ y las observaciones del cluster $H$:

   \begin{align}
     d_{CL}(G, H)= max_{i\in G,\ i'\in H} d_{ii'} 
   \end{align}

   En otras palabras, funciona combinando clusters en función de los puntos más alejados entre los dos clusters.

##### Ejemplo:

|   | 1  | 2  | 3 | 4 | 5 |
|---|----|----|---|---|---|
| **1** | 0  |    |   |   |   |
| **2** | 9  | 0  |   |   |   |
| **3** | 3  | 7  | 0 |   |   |
| **4** | 6  | 5  | 9 | 0 |   |
| **5** | 11 | 10 | 2 | 8 | 0 |

## DBSCAN

- DBSCAN (por su nombre en inglés *Density-based spatial clustering of applications with noise*) agrupa los datos en función de las densidades de las observaciones, mientras maneja el ruido de manera eficiente.  


- DBSCAN  incorpora  la noción de densidad. Si hay grupos de puntos de datos que existen en el mismo vencindario, estos se pueden ver como miembros del mismo cluster.


- Pero para hacerlo dependende de principalmente de 2 parámetros: *eps* y *min_samples*.


- **Formalmente** *Cluster.* Sea $D$ conjunto de puntos. El cluster $C$ con respecto a `eps` y `min_samples` es un subconjunto no vacío de $D$ que satisface las siguientes condiciones:
    1. $\forall\ p, q:$ if $p\in C$ y $q$ es alcanzable por densidad por $p$ con respecto a `eps` y `min_samples` dados, entonces $q \in C$.
    2. $\forall\ p, q \in C:$ $p$ está conectado por densidad con $q$ con respecto al `eps` y `min_samples` dados.
  
  - *Ruido.* Sean $C_1, \cdots, C_k$ los clústeres conformados a partir de los puntos  $D$ usando los parámetros `eps` y `min_samples` fijos. Definimos como ruido a todos los puntos que no pertenecen a ningún cluster pero están presentes en $D$:

$$ruido = \{p\in D | \forall i: p \notin C_i \}$$

Veamos como funciona:

<div style="max-width:400px">
    <img src = "figs/DBSCAN_tutorial.gif" />
</div>

## Clustering para "Marketing Data Science"

Para que un negocio prospere, es fundamental atraer nuevos clientes, al mismo tiempo que reteniendo efectivamente los actuales. 

El análisis de clusters facilita identificar grupos homogéneos permitiendo realizar estrategias que permiten captar oportunidades que los análisis tradicionales a menudo no son capaces.

En esta tarea es importante que los segmentos identificados mediante el análisis de clusters deben ser tanto reconocibles como accesibles. Esto permite a los gerentes de marketing personalizar productos y mensajes de manera eficaz, optimizando las campañas de marketing para alcanzar de manera efectiva a cada grupo.

Para ello es crucial elegir variables que no solo segmenten efectivamente el mercado, sino que también sean fáciles de medir y estén ampliamente disponibles. 


### "Marketing Bancario"

Esta base de datos proviene del UCI Machine Learning Repository y fue utilizada en estudios sobre campañas de marketing directo, particularmente en la promoción de depósitos a plazo fijo.

Los datos fueron recopilados de campañas de marketing telefónico de un banco en Portugal. El objetivo principal era predecir si un cliente suscribiría o no un depósito a plazo después de ser contactado. Pero nosotros la utilizaremos para ver si podemos identificar segmentos de clientes potenciales.

### Carga de datos 

In [ ]:
# Importar las librerías
import pandas as pd


# leer los datos
bank = pd.read_csv('data/bank_es.csv', sep = ',')

# Ver las primeras observaciones
bank.head()


1. **Variables Demográficas**:
     - `edad`: Edad del cliente.
     - `trabajo`: Tipo de empleo del cliente (e.g., "admin.", "blue-collar", "technician").
     - `estado civil`: Estado civil del cliente (e.g., "single", "married", "divorced").
     - `educación`: Nivel educativo del cliente (e.g., "primary", "secondary", "tertiary").

2. **Historial Bancario**:
     - `impago`: Si el cliente tiene crédito en mora (binaria: "sí", "no").
     - `saldo`: Saldo promedio en la cuenta bancaria del cliente.
     - `hipoteca`: Si el cliente tiene un préstamo hipotecario (binaria: "sí", "no").
     - `préstamo`: Si el cliente tiene un préstamo personal (binaria: "sí", "no").
     - `contactos_campaña`: Número de contactos realizados durante esta campaña.
     - `dias_ultimo_contacto`: Días transcurridos desde que el cliente fue contactado por última vez en una campaña anterior.
     - `contactos_previos`: Número de contactos realizados antes de esta campaña.
     - `resultado de la campaña previa`: Resultado de la campaña anterior (e.g., "success", "failure").

3. **Información del Contacto**:
     - `contacto`: Tipo de contacto de la comunicación (e.g., "cellular", "telephone").
     - `mes`: Último mes de contacto en la campaña.
     - `día`: Último día del mes en que se realizó el contacto.
     - `duración`: Duración de la última llamada en segundos.

4. **Variable de Resultado**:
     - `respuesta`: Variable binaria objetivo que indica si el cliente suscribió o no un depósito a plazo (binaria: "sí", "no").

Nos preocuparan principalmente aquellos individuos sin contactos previos a esta campaña.

In [ ]:
# Sin contacto previo
bank = bank[bank['contactos_previos'] == 0]

En este ejercicio  nos centramos principalmente en las variables demográficas (`edad`, `trabajo`, `estado civil`, `educación`) y algunas relacionadas a componentes bancarios (`hipoteca`, `préstamo`) que son accesibles para todos los clientes, incluidas aquellas personas que aún no tienen un historial de transacciones con el banco.



Vamos a preparar los datos para el análisis de clustering, se transforman las variables categóricas en variables binarias y se estandarizan las variables numéricas, lo que facilita la identificación de segmentos de clientes similares.


In [ ]:
# generar una nueva variable empleo_cuello_blanco que son los que tienen trabajos administrativos, gerenciales, emprendedores y autonomos
bank['empleo_cuello_blanco'] = bank['trabajo'].apply(lambda x: 1 if x in ['administrativo.', 'gerente', 'emprendedor', 'autonomo'] else 0)

# generar tabla de contingencia trabajo vs empleo_cuello_blanco
pd.crosstab(index = bank['trabajo'], columns = bank['empleo_cuello_blanco'])


In [ ]:
# generar una nueva variable empleo_cuello_azul que son los que tienen trabajos obreros, ama de casa, servicios, tecnico 
bank['empleo_cuello_azul'] = bank['trabajo'].apply(lambda x: 1 if x in ['obrero', 'ama_de_casa', 'servicios', 'tecnico'] else 0)

# generar tabla de contingencia trabajo vs empleo_cuello_blanco
pd.crosstab(index = bank['trabajo'], columns = bank['empleo_cuello_azul'])


In [ ]:
# generar dummies estado_civil en la base de datos
bank = pd.get_dummies(bank, columns = ['estado_civil'], drop_first = True)


In [ ]:
# generar dummies educacion en la base de datos
bank = pd.get_dummies(bank, columns = ['educacion'], drop_first = True)

In [ ]:
bank.head()

In [ ]:
# reemplazar hipoteca y prestamo "no" por 0 y "si" por 1
bank['hipoteca'] = bank['hipoteca'].apply(lambda x: 0 if x == 'no' else 1)
bank['prestamo'] = bank['prestamo'].apply(lambda x: 0 if x == 'no' else 1)

In [ ]:

# retener `edad`, `trabajo`, `estado civil`, `educación`, `hipoteca`, `préstamo`
bank = bank[['edad', 'empleo_cuello_blanco', 'empleo_cuello_azul', 'estado_civil_soltero', 'estado_civil_divorciado', 'educacion_primaria', 'educacion_secundaria', 'educacion_terciaria', 'hipoteca', 'prestamo']]
bank.head()

## Clustering Jerárquico

In [ ]:
# Importamos  las librerías
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.cluster.hierarchy import dendrogram

In [ ]:
from sklearn.preprocessing import scale
import numpy as np
import matplotlib.pyplot as plt

distances1 = linkage(scale(bank), method='complete', metric="euclidean")

In [ ]:

fig, ax = plt.subplots(figsize=(12, 5))
d = dendrogram(distances1, show_leaf_counts=True, leaf_font_size=6, ax=ax,labels=bank.index)
ax.set_xlabel('Observaciones', fontsize=6)
ax.set_yticks(np.arange(0, 10, 1))
ax.set_ylabel('Distancia', fontsize=14)
plt.show()


In [ ]:
distances2 = linkage(bank, method='complete', metric="euclidean")

In [ ]:


fig, ax = plt.subplots(figsize=(12, 5))
d = dendrogram(distances2, show_leaf_counts=True, leaf_font_size=6, ax=ax,labels=bank.index)
ax.set_xlabel('Observaciones', fontsize=6)
ax.set_yticks(np.arange(0, 20, 1))
ax.set_ylabel('Distancia', fontsize=14)
plt.show()


In [ ]:
distances3 = linkage(bank, method='complete', metric="cityblock")

In [ ]:

fig, ax = plt.subplots(figsize=(12, 5))
d = dendrogram(distances3, show_leaf_counts=True, leaf_font_size=14, ax=ax,labels=bank.index)
ax.set_xlabel('Observaciones', fontsize=14)
ax.set_yticks(np.arange(0, 20,1))
ax.set_ylabel('Distancia', fontsize=14)
plt.show()

## DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors = 4)
nbrs = neigh.fit(bank)
distancias, indices = nbrs.kneighbors(bank)
distancias = np.sort(distancias.flatten())
fig=plt.figure(figsize=(10,8), dpi= 100, facecolor='w', edgecolor='k')
plt.axhline(y = 0.5, color = 'r', linestyle = '--')
plt.plot(distancias)

In [ ]:
from kneed import KneeLocator

i = np.arange(len(distancias))
knee = KneeLocator(i, distancias, S=1, curve='convex', direction='increasing', interp_method='polynomial')

print(distancias[knee.knee])

In [ ]:
db = DBSCAN(eps=1, min_samples=30)
clusters=db.fit_predict(bank)


In [ ]:
bank['cluster_dbscan'] = clusters
bank.groupby('cluster_dbscan').mean()

# Referencias

- Leskovec, J., Rajaraman, A., & Ullman, J. D. (2020). Mining of massive data sets. Cambridge university press.
- Sheehan, D. (2022). https://dashee87.github.io/data%20science/general/Clustering-with-Scikit-with-GIFs/
- Waggoner, P. Unsupervised Machine Learning for Clustering in Political and Social Research. Mimeo

# Información de Sesión

In [ ]:
import session_info

session_info.show(html=False)